In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score)
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.decomposition import PCA
from sklearn.compose import TransformedTargetRegressor
from custom_transfomers.date_window import TimeWindowTransformer
from sklearn.dummy import DummyRegressor
from project_utils.data_manipulation import generate_aggregation
from sklearn.metrics import make_scorer
import pandas
from data_base.connection import session
from data_base.models import models
from sqlalchemy import select
from IPython import get_ipython

In [2]:
## Construção do dataframe utilizando buscas no banco de dados sql
%load_ext autoreload
%autoreload 2

In [3]:
query = select(
    models.Variables.date,
    models.Variables.precipitation.label('precipitation'),
    models.Variables.temperature.label('temperature'),
    models.Variables.evaporation.label('evaporation'),
    models.Variables.surface_runoff.label('surface_runoff'),
    models.Coordinate.river_id.label('river'),    
    models.Reservoir.level,
    models.Reservoir.streamflow
).\
    join(models.Variables.coordinate).\
    join(models.Reservoir, models.Variables.date == models.Reservoir.date)

RawDataFrame = pandas.read_sql(query, session.bind)




In [4]:
# DataFrame consolidado porém com os atributos para cada rio posicionados em uma diferente coluna
ConsolidatedDataFrame = (
    RawDataFrame.
    groupby(['date', 'river', 'level', 'streamflow']).
    agg({
        'precipitation': 'sum',
        'evaporation': 'sum',
        'temperature': 'mean',
        'surface_runoff':'mean',
    }).
    reset_index().
    pivot(index=["date", 'level', 'streamflow'], columns="river")
)

ConsolidatedDataFrame.insert(0,'previous_streamflow', pandas.DataFrame(ConsolidatedDataFrame.index.get_level_values('streamflow')).shift(1).values)
ConsolidatedDataFrame.insert(0,'previous_level', pandas.DataFrame(ConsolidatedDataFrame.index.get_level_values('level')).shift(1).values)

ConsolidatedDataFrame = ConsolidatedDataFrame.dropna()


In [5]:
seed = 0
scorer = make_scorer(accuracy_score)

rivers = session.query(models.River).all()

precipitation_agg = generate_aggregation('sum', 'precipitation', [river.id for river in rivers])
evaporation_agg = generate_aggregation('sum', 'evaporation', [river.id for river in rivers])
temperature_agg = generate_aggregation('mean', 'temperature', [river.id for river in rivers])
runoff_agg = generate_aggregation('mean', 'surface_runoff', [river.id for river in rivers])

cols = ['precipitation', 'evaporation', 'temperature', 'surface_runoff']

agg = precipitation_agg
agg.update(evaporation_agg)
agg.update(temperature_agg)
agg.update(runoff_agg)


In [9]:
grid_search_params = dict(
    estimator=Pipeline([
                ('windowing', TimeWindowTransformer(columns=cols)),
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', MinMaxScaler(feature_range=(0, 1))),
                ('clf', DummyRegressor())
            ]), 
            param_grid=[
                {
                    'windowing__aggregate': [agg],
                    'windowing__rolling': range(1, 32, 10),
                    'windowing__dropna': [False],
                    'clf': (
                        TransformedTargetRegressor(
                        transformer=MinMaxScaler(feature_range=(0, 1)), 
                        regressor=SVR(cache_size=1000)
                    ),),
                    'clf__regressor__C': range(1, 15, 3),
                    'clf__regressor__gamma': ['auto', 'scale'],
                    'clf__regressor__kernel': ['rbf']
                },
                {
                    'windowing__aggregate': [agg],
                    'windowing__rolling': range(1, 32, 10),
                    'windowing__dropna': [False],
                    'clf': (RandomForestRegressor(), ),
                    'clf__random_state': [seed],
                    'clf__n_estimators': [200]
                },
                {
                    'windowing__aggregate': [agg],
                    'windowing__rolling': range(1, 32, 10),
                    'windowing__dropna': [False],
                    'clf': (DecisionTreeRegressor(), ),
                    'clf__random_state': [seed]
                },
                {
                    'windowing__aggregate': [agg],
                    'windowing__rolling': range(1, 32, 10),
                    'windowing__dropna': [False],
                    'clf': (StackingRegressor(
                        estimators=[('RandomForest', RandomForestRegressor()), ('SVR', SVR())], 
                        final_estimator=RidgeCV()
                    ),),
                    'clf__RandomForest__random_state': [seed]
                }
            ],
            scoring='neg_root_mean_squared_error',
            cv=10,
            n_jobs=-1,
            verbose=10,
            error_score='raise'
)

In [11]:
targets = ['level', 'streamflow']

clf_search = {target: GridSearchCV(**grid_search_params) for target in targets}

In [12]:
level_classifier = clf_search['level']

df = pandas.DataFrame()
df['level'] = ConsolidatedDataFrame.index.get_level_values('level')
level_classifier.fit(ConsolidatedDataFrame, ConsolidatedDataFrame.index.get_level_values('level'))
df['p_level'] = level_classifier.predict(ConsolidatedDataFrame)
pandas.DataFrame(streamflow_classifier.cv_results_)



Fitting 10 folds for each of 52 candidates, totalling 520 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__regressor__C,param_clf__regressor__gamma,param_clf__regressor__kernel,param_windowing__aggregate,param_windowing__dropna,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.168517,0.027704,0.090684,0.022336,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.398794,-1.344635,-0.557179,-0.794668,-1.546368,-1.647113,-1.485091,-1.092766,0.429984,17
1,0.144194,0.015594,0.074279,0.013512,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.701475,-1.487738,-0.542268,-0.769409,-1.634953,-1.814281,-1.645113,-1.207802,0.485647,27
2,0.136018,0.006395,0.066820,0.007415,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.810438,-1.512826,-0.528760,-0.875910,-1.657302,-2.102396,-1.778361,-1.298773,0.531618,32
3,0.124147,0.016734,0.071384,0.011291,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.968147,-1.503822,-0.539900,-0.877758,-1.704857,-2.314831,-1.821593,-1.358391,0.586763,37
4,0.116896,0.016850,0.074872,0.013990,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.581313,-1.380143,-0.461769,-1.037982,-1.488679,-1.822807,-1.318802,-1.097358,0.463624,22
5,0.119105,0.013004,0.064855,0.007458,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.849613,-1.949324,-0.415201,-1.539528,-1.690925,-2.590147,-1.822039,-1.408961,0.681391,42
6,0.118363,0.008535,0.070357,0.008378,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.972479,-1.896514,-0.386408,-1.791435,-1.727946,-3.445595,-1.980596,-1.569701,0.859583,47
7,0.131293,0.022141,0.080028,0.010419,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-2.036351,-1.836700,-0.421838,-1.615456,-1.892809,-3.895114,-1.973790,-1.661120,0.941031,48
8,0.130734,0.022450,0.079818,0.011207,TransformedTargetRegressor(regressor=SVR(cache...,4,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.201918,-1.324976,-0.539706,-0.800318,-1.532912,-1.600103,-1.441520,-1.050969,0.413163,16
9,0.123428,0.014273,0.072316,0.009698,TransformedTargetRegressor(regressor=SVR(cache...,4,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-1.642173,-1.452975,-0.530938,-0.732161,-1.614919,-1.803023,-1.619453,-1.181045,0.483432,26


In [13]:
streamflow_classifier = clf_search['streamflow']

streamflow_classifier.fit(ConsolidatedDataFrame, ConsolidatedDataFrame.index.get_level_values('streamflow')) 
df['streamflow'] = ConsolidatedDataFrame.index.get_level_values('streamflow')
df['p_streamflow'] = streamflow_classifier.predict(ConsolidatedDataFrame)
pandas.DataFrame(streamflow_classifier.cv_results_)


Fitting 10 folds for each of 52 candidates, totalling 520 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__regressor__C,param_clf__regressor__gamma,param_clf__regressor__kernel,param_windowing__aggregate,param_windowing__dropna,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.159777,0.015905,0.080628,0.021644,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-278.743777,-321.122115,-267.083156,-187.006275,-337.991478,-330.871412,-355.546092,-301.255185,58.736667,31
1,0.145231,0.022370,0.076438,0.017125,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-238.002786,-336.124678,-262.229499,-168.191080,-336.259192,-348.652460,-327.125302,-291.551888,64.865527,27
2,0.140698,0.019429,0.072553,0.006710,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-221.996360,-328.477933,-264.499376,-156.437041,-328.744408,-368.154162,-301.770014,-282.782908,70.161971,25
3,0.145362,0.018041,0.070916,0.005718,TransformedTargetRegressor(regressor=SVR(cache...,1,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-207.457475,-330.749778,-272.505968,-161.519607,-325.788645,-379.278048,-283.320153,-284.683383,72.863084,26
4,0.158036,0.014412,0.080917,0.020152,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-350.861203,-322.010395,-279.373843,-242.794874,-321.363452,-305.624469,-325.088448,-314.875048,46.496427,34
5,0.156036,0.019452,0.078017,0.021029,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-290.867959,-364.458997,-295.190426,-276.074254,-317.496654,-370.764375,-312.634811,-322.456999,45.587322,38
6,0.157235,0.013859,0.075517,0.012858,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-241.147183,-346.356470,-305.708576,-262.202526,-326.667460,-353.355425,-319.345395,-315.103594,55.948550,35
7,0.160735,0.020893,0.069616,0.005573,TransformedTargetRegressor(regressor=SVR(cache...,1,scale,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-272.350869,-345.922033,-315.408552,-247.026754,-325.700405,-419.958623,-328.946228,-337.719544,64.740695,42
8,0.138831,0.011748,0.067315,0.008003,TransformedTargetRegressor(regressor=SVR(cache...,4,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-242.751048,-306.271178,-238.215276,-178.598271,-320.105197,-297.819939,-318.226632,-276.451798,56.623554,24
9,0.145933,0.017487,0.071715,0.008512,TransformedTargetRegressor(regressor=SVR(cache...,4,auto,rbf,"{('precipitation', 1): 'sum', ('precipitation'...",False,...,-211.874201,-313.626719,-246.116200,-177.188696,-316.744125,-319.779540,-291.962941,-271.784444,60.381519,13


In [14]:
df

,level,p_level,streamflow,p_streamflow
0,560.28,560.23780,1128.00,1269.16280
1,560.35,560.37935,1021.00,1035.36520
2,560.47,560.44180,1307.00,1204.15650
3,560.62,560.56080,1481.00,1462.76465
4,560.78,560.76405,1555.00,1602.96905
...,...,...,...,...
2502,568.51,568.53030,1068.11,1113.21135
2503,568.44,568.46345,1027.44,1034.77860
2504,568.40,568.40105,902.97,994.86475
2505,568.38,568.38940,1106.56,1085.02390
